In [ ]:
import sys
import os

# Add the project root to the Python path
package_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(f"Adding package root to sys.path: {package_root}")
if package_root not in sys.path:
	sys.path.append(package_root)


notebook_dir = os.getcwd()
print(f"Current notebook directory: {notebook_dir}")
# change to the directory to the root of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
print(f"Project root: {project_root}")
os.chdir(project_root)

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
print(os.environ["LANGCHAIN_TRACING_V2"])

In [ ]:
from langchain_core.messages import HumanMessage
import importlib

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import lets_talk.chains as chains
import lets_talk.prompts as prompts

In [ ]:
from lets_talk.config import Configuration

congiruation = Configuration()

print(Configuration.react_agent_prompt)

In [ ]:
import datetime
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
prompt =  ChatPromptTemplate.from_messages( [SystemMessagePromptTemplate.from_template(prompts.REACT_AGENT_PROMPT),MessagesPlaceholder(variable_name="history"), HumanMessagePromptTemplate.from_template("{input}")])

time = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()

a = prompt.invoke({"system_time": time,"history": ["A","B"], "input": "hello"})
print(a)

# from lets_talk.config import LLM_MODEL
# from langchain.chat_models import init_chat_model
# b = (prompt |  init_chat_model(LLM_MODEL)).invoke({"system_time": time,"history": [], "input": "What is current time in CST?"})
# print(b)


In [ ]:
# hot reload the module

importlib.reload(chains)
importlib.reload(prompts)


In [ ]:
#display marked down
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#printmd(r.content)

In [ ]:
import lets_talk.agent as agent


In [ ]:
importlib.reload(agent)


In [ ]:

uncompiled_graph = agent.build_graph()
graph =  uncompiled_graph.compile()

#show the graph

print(graph.get_graph().draw_ascii())

In [ ]:
from langchain_core.runnables import RunnableConfig
config = RunnableConfig(
        configurable={"user_id": "abc", }
    )

In [ ]:
# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_png()))
# from lets_talk.config import Configuration
# config = Configuration()
# a= await agent.check_question_tone({"question": "Give me latest?"},config=config)

# print(a)


a = await graph.ainvoke({"question": "Go to hell?"},config=config)
print(a)

In [ ]:
a["messages"][-2].content

In [ ]:
a["is_rude"]

In [ ]:
b = await graph.ainvoke({"question": "What is current time in CDT?"},config=config)
print(b)

In [ ]:
b["messages"][-1].content

In [ ]:
b["messages"]

In [ ]:
b["documents"]

In [ ]:
b['queries']

In [ ]:
d = await graph.ainvoke({"question": "What is data value chain?"},config=config)

In [ ]:
printmd(d["messages"][-1].content)

In [ ]:
from lets_talk.utils import format_docs
# write format_docs_v2 function
def format_docs_v2(docs):
    """
    Format the documents for display.
    """
    # Render all items from metadata in K:V format
    # and the page content in a single line
    # with a line break between each document
    # and a line break between each metadata item
    # and the page content in a single line
    
    formatted_docs = []
    for doc in docs:
        # Format all metadata as key-value pairs
        metadata_parts = []
        for key, value in doc.metadata.items():
            metadata_parts.append(f"**{key}:** {value}")
        
        # Join metadata items with line breaks
        metadata_str = "\n".join(metadata_parts)
        
        # Add page content with a line break after metadata
        formatted_doc = f"{metadata_str}\n\n**content:** {doc.page_content}"
        formatted_docs.append(formatted_doc)

    # Join all documents with double line breaks for separation
    return "\n\n---\n\n".join(formatted_docs)

# Use the corrected format_docs_v2 function
printmd(format_docs_v2(d["documents"]))

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
async for chunk,_ in graph.astream({"question": "Give me latest?"}, stream_mode="messages", config=config):
    print(chunk.content,end="")
    #print("\n")

#response

In [ ]:
async for chunk,metadata in graph.astream({"question": "What is data value chain?"}, stream_mode="messages"):
    if (
            chunk.content
            and not isinstance(chunk, HumanMessage)
            and metadata["langgraph_node"] == "agent"
        ):
        print(chunk.content,end="")
    #print("\n")

In [ ]:

uncompiled_graph = agent.build_graph()
graph =  uncompiled_graph.compile()
response = await graph.ainvoke({"question": "Greet the user and provide latest blog posts"},config=config)

In [ ]:
from lets_talk.utils import get_message_text
printmd(get_message_text(response["messages"][-1]))

In [ ]:
response =  await graph.ainvoke({"question": "Who are you?"},config=config)
printmd(response["messages"][-1].content)

In [ ]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
# Update the config dictionary to include recursion_limit
config_with_recursion = {**config, "recursion_limit": 2}

response = await graph.ainvoke({"question": "I'm Isa and my email is isa@thedataguy.pro, send a message to thedataguy that I like your blog."}, config=config_with_recursion)
printmd(response["messages"][-1].content)